# Introduction to the Prediktor Edge Client python library

* TOC

## Background

The PyPrediktorEdgeClient library allows the user to connect to an Apis instances. It is possible to create and manipulate Apis modules and items. The following core operations are available

* Apis Hive Service instances
  * Creating an Apis Hive service
  * Starting and stopping an Apis Hive service
  * Deleting an Apis Hive service
* Apis Hive operations
  * Creating modules
  * Listing Apis modules
  * Reading data as VQT records
* Apis Hive Module operations
  * Accessing module properties
  * Setting/getting module properties directly
  * Listing and retrieving module items
* Apis Hive Item operations
  * Accessing item attributes
  * Setting/getting module attributes directly


## Installation

```
pip install git+https://github.com/PrediktorAS/PyPrediktorEdgeClient.git
```

See the readme.md as well as [The PythonNet Github page](https://github.com/pythonnet/pythonnet/wiki/Troubleshooting-on-Windows,-Linux,-and-OSX) for further installation instructions.

## Library structure

The library is structured as a package with several subpackages:

    pyprediktoredgeclient
        hive
        hiveservices
        util
        honeystore^*
        timeseries^*
        chronicle^*

*) Not completed yet

## API conventions

There are several conventions used in the design of the *Python interface for Apis Edge*, `pyprediktoredgeclient`. The API follows the conventions set out in
[PEP8](https://www.python.org/dev/peps/pep-0008/), but with relaxation regarding line length.

* Variable, function and method names uses `snake_case` -- lower case names joined with underscore
* Classes are CamelCase
* Method names are structured as *verb_subject*, i.e. `get_item('')`
* Attributes (as python language construct, not to be confused with item attributes) are constructed from subjects or adjectives (i.e. `hive.module_types`).

# Hive services

Hive services is concerned with the representation of Hive instances on the computer, not the implementation of logic on that Hive instance.

## Importing the module

The hive instances are accessed through the `hiveservices` module

In [1]:
import sys
sys.path.append( r'C:\Users\LocalUser\Source\Repos\PyPrediktorEdgeClient')
from pyprediktoredgeclient import hiveservices

##  List all hive instances UUIDs on the machine

In [2]:
uuids = hiveservices.instance_identifiers()
uuids

[UUID('51f92300-ca68-11d2-85c3-0000e8404a66'),
 UUID('04d4eb61-ccd3-43cc-a4ca-d2b5c4c56b05'),
 UUID('72dfd4a1-f484-4e8c-9818-64e17c7ddbaf'),
 UUID('8322407c-ea3d-4886-9d99-cc8ccd35da6f'),
 UUID('8840f50b-924d-4429-bc52-d67840169db8'),
 UUID('f84849c5-e05e-4666-a290-5792b0448a50')]

## List all instances 

The `hiveservices.list_instances()`function returns a list of `HiveInstance` objects. The main properties of a `HiveInstance` object is:

* `name`: The name of the instance
* `prog_id`: The prog id (Com class name) of the Hive instance
* `running`: A bool property indicating wheter the instance is running or not
* `CLSID`: The UUID class id.

In [3]:
instance_list = hiveservices.list_instances()

for inst in instance_list:
    print(f"{inst.name}\n\t{inst.running}\t{inst.CLSID}\t{inst.prog_id}\n")

ApisHive
	False	51f92300-ca68-11d2-85c3-0000e8404a66	Prediktor.ApisLoader.1

ScadaInput
	False	04d4eb61-ccd3-43cc-a4ca-d2b5c4c56b05	Prediktor.ApisLoader.ScadaInput

pytestinstance
	True	72dfd4a1-f484-4e8c-9818-64e17c7ddbaf	Prediktor.ApisLoader.pytestinstance

Kontornett
	False	8322407c-ea3d-4886-9d99-cc8ccd35da6f	Prediktor.ApisLoader.Kontornett

AggregatedTags
	False	8840f50b-924d-4429-bc52-d67840169db8	Prediktor.ApisLoader.AggregatedTags

Simulator
	False	f84849c5-e05e-4666-a290-5792b0448a50	Prediktor.ApisLoader.Simulator



check that the list of UUID's and instance_list are same length and that all UUIDs are available

In [4]:
assert len(uuids) == len(instance_list)

#check that all instances in instance list has a corresponding UUID
for instance in instance_list:
    assert instance.CLSID in uuids

## Creating a new instance

We'll try to fetch the instance 'pytest' and create it if it doesn't exist

In [5]:

try:
    pytestinstance = hiveservices.get_instance('pytestinstance')
except hiveservices.Error:
    pytestinstance = hiveservices.add_instance('pytestinstance')

# check that 'pytest' is among the instances

assert 'pytestinstance' in [instance.name for instance in hiveservices.list_instances()]

In [6]:
dir(pytestinstance)

['CLSID',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_running',
 '_set_running',
 'api',
 'is_default',
 'name',
 'prog_id',
 'running']

## Starting and stopping services

The running state of Hive instances can be interrogated through the `.running` attribute of an `ApisInstance` object

In [7]:
if pytestinstance.running:
    print("'pytestinstance' instance is running")
else:
    print("'pytestinstance' instance was stopped, starting")

pytestinstance.running = True    

'pytestinstance' instance is running


# Hive, Modules and Items

The `hive` subpackage gives the user access to the Apis Edge hive and related parts such as modules and items. The `hive` functionality must not be confused with `hiveservices` mentioned above. The classes and functions in `hive` is concerned with the configuration of the system whereas `hiveservice` deals with the management of the hive on the computer, such as stopping and starting the system.

Some clarifications and definitions might be in order. The main classes defined in the `hive`package are:

* `Hive` This class contains the configuration of one hive. A hive is an insulated service that manages input, output, logging and processing. A `Hive` contains any number of `Module`s.
* `Module` This class contains all the `Item`s associated with one functionality. A functionality could be the OPC connection to a particular server. The parameters that control the `Module` behaviour are called `Property`. 
* `Property` The parameters for one `Module` such as ExchangeRate.
* `Item` contains the real-time information in the system. The various parameters and values associated with one `Item` are known as `Attribute`s

## Importing the package and connecting to a Hive

In this example we use the `hiveinstance` object from the previous section to connect to the `hive` however we could also use the prog-id of the hive as a connection parameter, 

In [8]:
from pyprediktoredgeclient import hive
test_hive = hive.Hive(pytestinstance)           #alternative: test_hive = hive.Hive('Prediktor.ApisLoader.pytestinstance')
assert test_hive.name=='pytestinstance'

## The *module types*

A hive will support several *module types*. Each module type takes care of a given aspect of the Hive configuration, an *OPC-UA client module* will used to connect to a remote computer, a *Worker module* will be used to perform calculations etc. We can retrive the supported module types with the `.module_types` attribute of the hive object.

In [9]:
mod_class_names = [mod_type.class_name for mod_type in test_hive.module_types]
assert 'ApisWorker' in mod_class_names

Lets print out the first ten module class names

In [10]:
print('\n'.join(mod_class_names[:10]))
worker_type = test_hive.get_module_type('ApisWorker')

ApisOPC
ApisTaskScheduler
ApisHAGovernor
ApisBatchOptimizerBee
ApisIntegoBee
ApisMessageBuilderBee
ApisModbusSlave
ApisLVEstimator2
ApisGPSolarBee
ApisHSMirror


## Accessing, adding and removing modules from the hive.

The modules in the hive can be retrived directly by name or numeric index by subscripting, (i.e. `mod = hive['mymodule']`) or by iterating over the hive (i.e. `for mod in hive: ...`). Modules are added and removed from a hive using `.add_module(module_type, module_name)` and `del hive['module_name']` respectivly

In [11]:
for mod in test_hive.modules:
    if "testworker" in mod.name:
        del test_hive[mod.name]

module_names = [mod.name for mod in test_hive]

creating two modules. We either use a `ModuleType` instance or a string as the module_type parameter

In [12]:
test_module= test_hive.add_module(worker_type, 'testworker')
test_module2 = test_hive.add_module('ApisWorker', 'testworker2')

test_modules = [mod for mod in test_hive if 'testworker' in mod.name]
assert len(test_modules)==2

## Setting module properties

The module properties can be accessed directly

In [13]:
test_module.ExchangeRate=1000

# Getting available item types



In [14]:
item_type_names = [item_type.Name for item_type in test_module.item_types]

assert 'Signal' in item_type_names
assert 'Variable' in item_type_names
assert 'Function item' in item_type_names
item_type_names

['Signal',
 'Time',
 'Variable',
 'VariableVector',
 'BitSelect',
 'String formatter',
 'Multiplexer',
 'TrigEvtBrokerCmd',
 'VariableMatrix',
 'Expression',
 'ExtractVector',
 'Module state items',
 'Item attribute items',
 'Module events items',
 'Function item']

# Adding and accessing items

In [15]:
func_item_type = test_module.get_item_type('Function item')

var1 = test_module.add_item('Variable', 'var1')
var2 = test_module.add_item('Variable', 'var2')
sig1 = test_module.add_item('Signal', 'sig1')
func = test_module.add_item(func_item_type, 'func1')


## Adding attributes

In [16]:
var1.add_attr("Text1", "Some text here")
var2.Text1 = "Some other text"

assert var2.Text1=="Some other text"

## Setting External items

In [17]:
import time

func.Expression = "ex1+ex2"
func.external_items = [var1, var2]
var1.Value = 3.0
var2.Value = 5.5

time.sleep(1.0)     #Let APIS propagate the values
assert func['Value'] == 8.5

# hiveservices revisited - deleting a hive

In [20]:
pytestinstance.running = False

hiveservices.remove_instance(pytestinstance)

In [ ]:
from pyprediktoredgeclient import util
util.importlocation

'C:\\Program Files\\APIS\\Bin64'